# Adding parameters used in the synapsis physiology recipes

## Context


See ticket [BBPP134-233](https://bbpteam.epfl.ch/project/issues/secure/RapidBoard.jspa?rapidView=340&view=detail&selectedIssue=BBPP134-233)

## Imports

In [9]:
import os
import json
import re
import pandas as pd
import copy
import rdflib
import yaml
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
from rdflib.paths import OneOrMore

from bmo.ontologies import subontology_from_term
from bmo.utils import BMO, BRAIN_REGION_ONTOLOGY_URI, NSG, SCHEMAORG, SHACL, NXV

Read YAML files

In [2]:
with open('../data/synapse_types/E1_PT.yaml') as f:
    # use safe_load instead load
    e1_yaml = yaml.safe_load(f)

In [4]:
with open('../data/synapse_types/ProbAMPANMDA_EMS.yaml') as f:
    # use safe_load instead load
    probAMPA_yaml = yaml.safe_load(f)

In [3]:
e1_yaml

{'model_template': 'ProbAMPANMDA_EMS',
 'synapse_type': 'E1_PT',
 'name': 'Excitatory proximal targeting.',
 'description': 'Rare facilitating EXC - INH synapses: USE, D, F from Wang et al. 2006, dtc from Hausser and Roth 1997,  Hill coefs. from Rozov et al. 2001, NMDA ratio from Wang and Gao 2009, g_syns and NRRP: same as E2_NBC (for _PT) and E1_MC (for _DT). The GUDF parameters were set to the parameters of a TM model for electrophysiological traces.',
 'parameters': [{'name': 'tau_r_NMDA',
   'description': 'Rise time for NMDA currents.',
   'units': 'ms',
   'value': 0.29,
   'scope': 'GLOBAL',
   'functionalize': False},
  {'name': 'tau_d_NMDA',
   'description': 'Decay time for NMDA currents.',
   'units': 'ms',
   'value': 43.0,
   'scope': 'GLOBAL',
   'functionalize': False},
  {'name': 'tau_r_AMPA',
   'description': 'Rise time for AMPA currents.',
   'units': 'ms',
   'value': 0.2,
   'scope': 'GLOBAL',
   'functionalize': False},
  {'name': 'tau_d_AMPA',
   'description': '

In [6]:
combined_parameters = e1_yaml['parameters'] + probAMPA_yaml['parameters']

## Read parameter ontology

In [7]:
base = '/Users/cgonzale/Documents/code/brain-modeling-ontology/ontologies/bbp'

In [19]:
parameter_ontology = rdflib.Graph()
parameter_ontology.parse(f"{base}/parameters.ttl")

<Graph identifier=Nbb6cc73609924d7ea1c61e34158cc4df (<class 'rdflib.graph.Graph'>)>

In [20]:
def add_term(graph, new_s, label, units, description, parent_label=None):
    triples_to_add = set()
    if parent_label:
        for s, p, o in graph.triples((None, RDFS.label, rdflib.term.Literal(parent_label, lang='en'))):
            triples_to_add.add((new_s, RDFS.subClassOf, s))
    triples_to_add.add((new_s, RDFS.label, rdflib.term.Literal(label, lang='en')))
    triples_to_add.add((new_s, RDF.type, OWL.Class))
    triples_to_add.add((new_s, SCHEMAORG.unitCode, rdflib.term.Literal(units)))
    triples_to_add.add((new_s, SKOS.definition, rdflib.term.Literal(description, lang='en')))
    for el in triples_to_add:
        graph.add(el)

In [21]:
parent_label = "Model Brain Parameter"

In [22]:
done_parameters = []

In [23]:
for parameter in combined_parameters:
    name = parameter['name']
    print(name)
    description = parameter['description']
    units = parameter['units']
    iri = "https://bbp.epfl.ch/ontologies/core/parameters/"+name
    url = rdflib.URIRef(iri)
    if name not in done_parameters:
        add_term(parameter_ontology, url, name, units, description, parent_label)
        done_parameters.append(name)

tau_r_NMDA
tau_d_NMDA
tau_r_AMPA
tau_d_AMPA
Use
Fac
Dep
Nrrp
tau_r_AMPA
tau_d_AMPA
tau_r_NMDA
tau_d_NMDA
Use
Dep
Fac
Nrrp
conductance
NMDA_ratio


In [24]:
parameter_ontology.serialize(destination=f"{base}/parameters_updated.ttl")

<Graph identifier=Nbb6cc73609924d7ea1c61e34158cc4df (<class 'rdflib.graph.Graph'>)>